# Spotify Top Tracks Notebook
### *This notebook will document the process of calling the Spotify API and using the retrieved data to gain insight on listening habits.*

In [ ]:
# Import necessary dependencies
import requests
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
from tokens import get_access_token

## Retrieve Top Tracks

In [ ]:
# Access token from tokens.py
access_token = get_access_token()

headers = {
    'Authorization': f'Bearer {access_token}'
}

# Base url for all Spotify API calls
BASE_URL = 'https://api.spotify.com/v1/'

In [ ]:
r = requests.get(f'{BASE_URL}me/top/tracks',
                 headers=headers,
                 params={
                     'limit': 100,
                     'time_range': 'short_term',
                        })
r = r.json()


In [ ]:
tracks = {'id':[], 'track_title':[], 'artist':[], 'album':[], 'r_date':[]}
tracks_id = []
pos = 1
for item in r['items']:
    track_title = item['name']
    track_id = item['id']
    tracks_id.append(track_id)
    artists = []
    for i in range(0, len(item['artists'])):
        artists.append(item['artists'][i]['name'])
    artist = "/".join(artists)
    album = item['album']['name']
    r_date = item['album']['release_date']
    """print(f'{pos})\n\tTrack: "{track_title}", \n\tID: "{track_id}"\n\tArtist(s): {artist}\n\tAlbum: "{album}"\
        \n\tRelease Date: {r_date}\n')
    print('--------------------------------------------------------------------------------------\n')"""
    tracks['id'].append(track_id)
    tracks['track_title'].append(track_title)
    tracks['artist'].append(artist)
    tracks['album'].append(album)
    tracks['r_date'].append(r_date)
    pos+=1


In [ ]:
df_tracks = pd.DataFrame(tracks)
df_tracks

## Track Audio Features

In [ ]:
ids = ','.join(tracks_id)

In [ ]:
r2 = requests.get(f"{BASE_URL}audio-features",
                  headers=headers,
                 params={
                     'ids':ids,
                        })
r2 = r2.json()

df_feats = pd.DataFrame(r2['audio_features'])
df_feats.head(5)

In [ ]:
df = pd.concat([df_tracks, df_feats], axis=1)
df.drop(['id', 'type', 'uri', 'track_href', 'analysis_url'], axis=1, inplace=True)
df.head()

In [ ]:
from datetime import datetime

df.r_date = df.r_date.apply(lambda x: x[:7])
df.head()

## Plots/Machine Learning

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
df_1 = df[['track_title', 'energy', 'tempo']].sort_values('energy', axis=0)
df_1.head()

In [ ]:
sns.lineplot(data = df_1,
               x = 'energy',
               y = 'tempo')

In [ ]:
keymap = {0:'C', 1:'C#', 2:'D', 3:'D#/Eb', 4:'E', 5:'F', 6:'F#/Gb', 
          7:'G', 8:'G#/Ab', 9:'A', 10:'A#/Bb', 11:'B', 12:'C'}
modemap = {0:'minor', 1:'Major'}
df['key-mode'] = df['key'].apply(lambda x: keymap[x]) + '-' + df['mode'].apply(lambda x: modemap[x])
keymode = ['key', 'mode', 'key-mode']
df[keymode].head()

In [ ]:
df_2 = df.groupby(by=['artist'])
df_2